In [1]:
import sqlite3
from sqlite3 import Error

In [14]:
def create_connection():
    """ create a database connection to a SQLite database """
    conn = sqlite3.connect("raw_open_library/openlibrary.sqlite3")
    print(sqlite3.version)
    return conn

def create_tables(conn):
    curs = connection.cursor()
    curs.execute('''create table if not exists genres (genre_id integer primary key, description string)''')
    curs.execute('create table if not exists books (open_library_id integer primary key, title text, author text)')
    curs.execute('''
        create table if not exists books_to_genres (
            open_library_id integer,
            genre_id integer,
            foreign key(open_library_id) references books(open_library_id),
            foreign key(genre_id) references genres(genre_id)
        )
    ''')
    
with create_connection() as connection:
    create_tables(connection)

2.6.0
